In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np
import re
import tqdm
from data.make_data import get_links, is_article, save_links

np.random.seed(41)

In [ ]:
# Download latest version
path = kagglehub.dataset_download("bhavikjikadara/bbc-news-articles")

print("Path to dataset files:", path)

Path to dataset files: /home/vscode/.cache/kagglehub/datasets/bhavikjikadara/bbc-news-articles/versions/1


: 

In [ ]:
df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "adityakharosekar2/guardian-news-articles",
    'guardian_articles.csv'
)

In [ ]:
df.head()

,article_id,sectionName,webTitle,webUrl,bodyContent,webPublicationDate,id
0,us-news/2016/jan/31/iowa-caucus-underdog-candi...,US news,Iowa underdogs put on brave faces despite all ...,https://www.theguardian.com/us-news/2016/jan/3...,As polling day looms and the cameras turn only...,2016-01-31T23:53:37Z,1
1,us-news/2016/jan/31/iowa-caucus-worlds-most-pa...,US news,Iowa caucus: hologram eagle and Jesus star on ...,https://www.theguardian.com/us-news/2016/jan/3...,"In Des Moines on Sunday, the Guardian was give...",2016-01-31T23:46:28Z,2
2,world/2016/jan/31/tanzania-britsh-helicopter-p...,World news,British pilot in Tanzania 'manoeuvred ​to save...,https://www.theguardian.com/world/2016/jan/31/...,A British pilot who was shot dead by an elepha...,2016-01-31T23:43:48Z,3
3,football/2016/jan/31/late-winner-gets-usa-off-...,Football,USA 3-2 Iceland | International friendly match...,https://www.theguardian.com/football/2016/jan/...,USA took a step toward shaking off the ghosts ...,2016-01-31T23:30:49Z,4
4,football/2016/jan/31/blackburn-paul-lambert-ox...,Football,Reinvigorated Paul Lambert reflects after impr...,https://www.theguardian.com/football/2016/jan/...,"The clean-shaven, spectacle free and suspiciou...",2016-01-31T22:30:10Z,5


In [ ]:
df.to_csv('../data/raw/gurdian_articles.csv', index=False)

In [4]:
df = pd.read_csv('../data/raw/gurdian_articles.csv')

In [5]:
n = 1

get_links(df.sample(10).iloc[1,:][['webUrl', 'id']])

{'url': 'https://www.theguardian.com/world/2020/dec/23/lufthansa-fly-fresh-food-uk-shortage-fears-fruit-vegetables-border-french-blockade',
 'id': '114661',
 'links': [{'href': '#img-1', 'link': 'View image in fullscreen'},
  {'href': 'https://www.theguardian.com/world/coronavirus-outbreak',
   'link': 'Coronavirus'},
  {'href': 'https://www.theguardian.com/world/series/coronavirus-live/latest',
   'link': 'Coronavirus – latest updates'},
  {'href': 'https://www.theguardian.com/world/coronavirus-outbreak',
   'link': 'See all our coronavirus coverage'},
  {'href': 'https://www.theguardian.com/profile/sarahbutler',
   'link': 'Sarah Butler'},
  {'href': 'mailto:?subject=Supermarkets airlifting fruit and vegetables to UK amid shortage fears&body=https://www.theguardian.com/world/2020/dec/23/lufthansa-fly-fresh-food-uk-shortage-fears-fruit-vegetables-border-french-blockade?CMP=share_btn_url',
   'link': 'Share'},
  {'href': 'https://www.theguardian.com/world/2020/dec/23/uk-ferry-passenger

In [5]:
df.sample(10).iloc[1,:][['webUrl', 'id']]

webUrl    https://www.theguardian.com/politics/2021/apr/...
id                                                   122562
Name: 122561, dtype: object

In [6]:
res = get_links(df.sample(10).iloc[1,:][['webUrl', 'id']])

for l in res['links']:
    if is_article(l['href']):
        print(l['href'], l['link'])

https://www.theguardian.com/sport/blog Sportblog
https://www.theguardian.com/football/carabao-cup Carabao Cup
https://www.theguardian.com/profile/barneyronay Barney Ronay
https://www.theguardian.com/football/2021/apr/25/manchester-city-tottenham-carabao-cup-match-report Manchester City win fourth Carabao Cup in a row as Laporte sinks SpursRead more
https://www.theguardian.com/football/manchestercity Manchester City
https://www.theguardian.com/football/carabao-cup Carabao Cup
https://www.theguardian.com/football/2021/apr/24/guardiola-warned-citys-hierarchy-over-doomed-european-super-league Guardiola warned Manchester City hierarchy he would speak out over ESLRead more


In [10]:
save_links(res)

## Define a test set

In [5]:
df.sort_values('webPublicationDate',ascending=False,inplace=True)
# Sort to have most recent at the top

In [6]:
n_test = 100

df_test = df.iloc[:n_test]

In [6]:
test_set_links = df_test[['webUrl', 'id']].apply(get_links,axis=1)

In [7]:
def extract_links(blob):
    return list(filter(is_article, [d['href'] for d in blob['links']]))

In [8]:
def is_article_in_df(link):
    return link in df['webUrl'].values

In [11]:
is_article_in_df('https://www.theguardian.com/world/2024/jun/20/ukraine-war-russia-accused-of-using-banned-phosphorus-bombs-in-battle-for-bakhmut')

False

In [9]:
is_article_in_df(df.iloc[0]['webUrl'])


True

In [10]:
def print_status(test_set_links):
    status_string = 'Looking in {:d} articles, {:d} links to articles found, of which {:d} are unique {:.2f}%\n\
    And {:d} are already in DB ({:.2f}%)\n\
            {:d} left to do'\
    .format(test_set_links.shape[0], len(flat_list), len(flat_list_unique), (100 * (len(flat_list_unique) / float(len(flat_list)))),
            n_in_db, (100 * (n_in_db / float(len(flat_list_unique)))),(len(flat_list_unique) - n_in_db))
    print(status_string)


In [19]:
print_status(test_set_links)

Looking in 100 articles, 932 links to articles found, of which 679 are unique 72.85%
    And 159 are already in DB (23.42%)
            520 left to do


## Snowball Sample

In [8]:
df.tail()

,article_id,sectionName,webTitle,webUrl,bodyContent,webPublicationDate,id
1937,environment/andes-to-the-amazon/2016/jan/25/hi...,Environment,Hitler Rojas - the Peruvian farmer killed for ...,https://www.theguardian.com/environment/andes-...,Large hydro-electric power may be cleaner than...,2016-01-25T20:32:33Z,1938
1938,world/2016/jan/25/former-maldives-president-mo...,World news,Former Maldives president warns of return to d...,https://www.theguardian.com/world/2016/jan/25/...,The first democratically elected president of ...,2016-01-25T20:30:40Z,1939
1939,commentisfree/2016/jan/26/it-is-beyond-time-fo...,Opinion,It is beyond time for Britain to apologise to ...,https://www.theguardian.com/commentisfree/2016...,"Every year Australia Day gets bigger, more ost...",2016-01-25T20:30:04Z,1940
1940,football/2016/jan/25/west-ham-face-possibility...,Football,West Ham deny possibility of groundsharing at ...,https://www.theguardian.com/football/2016/jan/...,West Ham have denied claims from the London Le...,2016-01-25T20:22:58Z,1941
1941,business/nils-pratley-on-finance/2016/jan/25/g...,Business,"A 'major success' would be a fair tax system, ...",https://www.theguardian.com/business/nils-prat...,Even the government couldn’t sustain its boast...,2016-01-25T20:07:16Z,1942


In [11]:
from tqdm.auto import tqdm
tqdm.pandas()
test_set_links = df_test[['webUrl', 'id']].progress_apply(get_links,axis=1)

#df = pd.concat([df,pd.DataFrame(data = {'article_id' : 'XXX', 'sectionName' : 'XXX', 'webTitle' : 'XXX', 
#                    'webUrl' : test_set_links.apply(lambda x:x['url']), 'bodyContent' : 'XXX','webPublicationDate' : pd.NaT,'id' : 999999})])

while True:

    flat_list = [item for sublist in test_set_links.apply(extract_links) for item in sublist]

    flat_list_unique = list(set(flat_list))

    is_list_in_db = list(map(is_article_in_df, flat_list_unique))

    n_in_db = sum(is_list_in_db)

    ratio = n_in_db / float(len(flat_list_unique)) 
    
    print_status(test_set_links)
    
    # test_set_links = [l for l,flag in zip(test_set_links,is_list_in_db) if flag == False]
    # Only keep links to articles not yet in the DB
    
    flat_list_unique = [l for l,flag in zip(flat_list_unique,is_list_in_db) if flag == False]
    # Check this

    test_set_links = pd.Series(map(get_links,tqdm([{'webUrl':url, 'id' : -99999} for url in flat_list_unique])))

    # Add these parsed articles into the DB
    df = pd.concat([df,pd.DataFrame(data = {'article_id' : 'XXX', 'sectionName' : 'XXX', 'webTitle' : 'XXX', 
                    'webUrl' : test_set_links.apply(lambda x:x['url']), 'bodyContent' : 'XXX','webPublicationDate' : pd.NaT,'id' : 999999})])

    if ratio > 0.4:
        break

100%|██████████| 100/100 [00:05<00:00, 16.97it/s]


Looking in 100 articles, 932 links to articles found, of which 679 are unique 72.85%
    And 159 are already in DB (23.42%)
            520 left to do
flat_list_unique 520


100%|██████████| 520/520 [01:13<00:00,  7.04it/s]


Looking in 520 articles, 2909 links to articles found, of which 1434 are unique 49.30%
    And 404 are already in DB (28.17%)
            1030 left to do
flat_list_unique 1030


100%|██████████| 1030/1030 [01:45<00:00,  9.74it/s]


...

100%|██████████| 3094/3094 [05:26<00:00,  9.47it/s]


Looking in 3094 articles, 17312 links to articles found, of which 6898 are unique 39.85%
    And 868 are already in DB (12.58%)
            6030 left to do
flat_list_unique 6030


100%|██████████| 6030/6030 [10:36<00:00,  9.47it/s]


Looking in 6030 articles, 10921 links to articles found, of which 1969 are unique 18.03%
    And 962 are already in DB (48.86%)
            1007 left to do
flat_list_unique 1007


100%|██████████| 1007/1007 [01:38<00:00, 10.20it/s]


5 Iterations beyond seed set

100%|██████████| 100/100 [00:05<00:00, 16.97it/s]
Looking in 100 articles, 932 links to articles found, of which 679 are unique 72.85%
    And 159 are already in DB (23.42%)
            520 left to do
flat_list_unique 520
100%|██████████| 520/520 [01:13<00:00,  7.04it/s]
Looking in 520 articles, 2909 links to articles found, of which 1434 are unique 49.30%
    And 404 are already in DB (28.17%)
            1030 left to do
flat_list_unique 1030
100%|██████████| 1030/1030 [01:45<00:00,  9.74it/s]
Looking in 1030 articles, 7463 links to articles found, of which 3785 are unique 50.72%
    And 691 are already in DB (18.26%)
            3094 left to do
flat_list_unique 3094
100%|██████████| 3094/3094 [05:26<00:00,  9.47it/s]
Looking in 3094 articles, 17312 links to articles found, of which 6898 are unique 39.85%
    And 868 are already in DB (12.58%)
            6030 left to do
flat_list_unique 6030
100%|██████████| 6030/6030 [10:36<00:00,  9.47it/s]
Looking in 6030 articles, 10921 links to articles found, of which 1969 are unique 18.03%
    And 962 are already in DB (48.86%)
            1007 left to do
flat_list_unique 1007
100%|██████████| 1007/1007 [01:38<00:00, 10.20it/s]

In [17]:
df.to_csv('../data/interim/snowballed.csv')